# NameX Daily Stats

We need to load in these libraries into our notebook in order to query, load, manipulate and view the data

In [ ]:
import csv
from datetime import datetime, timedelta
from config import Config

%load_ext sql
%config SqlMagic.displaylimit = 5

This will create the connection to the database and prep the jupyter magic for SQL...

In [ ]:
%sql $Config.NAMEX_DATABASE_URI

Daily totals for specified date: Following query, 'current_date - 0' means today, 'current_date - 1' means yesterday, 'current_date - 2' means the day before yesterday...

In [ ]:
%%sql stat_daily_submitted  <<
SELECT count(r.*) AS "Number of Names Submitted:" FROM requests r
WHERE date(r.submitted_date at time zone 'utc' at time zone 'pst')  = date(current_date - 1)

In [ ]:
filename ='daily_totals_' + datetime.strftime(datetime.now()-timedelta(1), '%Y-%m-%d') +'.csv'

edt = stat_daily_submitted.DataFrame()

with open(filename, 'w') as f:
    edt.to_csv(f, sep=',', encoding='utf-8', index=False)

if edt.empty:
    with open(filename, 'a') as f:
        writer = csv.writer(f)
        writer.writerow(('No Data Retrieved',''))

In [ ]:
%%sql stat_daily_completed  <<
WITH Detail AS 
(
    SELECT to_char(date(current_date -1), 'YY-Mon-DD') AS EXAMINED_DATE
    , (SELECT username from users u where u.id=e.user_id) AS EXAMINER     
    , count(e.*) + count(r.*) FILTER (WHERE r.priority_cd = 'Y') AS TOTAL                  
    FROM events e,
    requests r
    WHERE e.user_id != 1
    AND r.id = e.nr_id
    AND date(e.event_dt at time zone 'utc' at time zone 'pst') = date(current_date - 1)
    AND e.state_cd in ('APPROVED','REJECTED','CONDITIONAL','CANCELLED')
    GROUP BY e.user_id 
    ORDER BY total DESC
)

SELECT * FROM Detail WHERE examiner LIKE '%idir%' 
UNION ALL
SELECT null, 'SUM' EXAMINER, sum(TOTAL) AS TOTAL FROM Detail WHERE examiner LIKE '%idir%'

In [ ]:
edt = stat_daily_completed.DataFrame()

if not edt.empty: 
    edt['examiner'] = edt['examiner'].str.replace('idir/','')

Save to CSV
    

In [ ]:
with open(filename, 'a') as f:      
    f.write('\n\n Number of Names Processed:\n')
    edt.to_csv(f, sep=',', encoding='utf-8', index=False) 

if edt.empty:
    with open(filename, 'a') as f:
        writer = csv.writer(f)
        writer.writerow(('No Data Retrieved',''))